### Load traning data and do one-hot encoding on labels

In [14]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('training_after_PCA_after_taking_4.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

# generate labels for the training 
integer_encoded = []
for i in range(9):
    integer_encoded += [i] * 30

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_train=onehot_encoded
x_train=data

### Load testing data and do one-hot encoding on labels

In [15]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('testing_after_PCA_after_taking_4.csv', newline='') as csvfile:
    data_test = list(csv.reader(csvfile))


# generate labels for the training 
integer_encoded = [0]*31 + [1]*35 + [2]*88 + [3]*44 + [4]*29 + [5]*24 + [6]*40 + [7]*50 + [8]*29

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_test=onehot_encoded
x_test=data_test
print(len(y_test))
print(len(x_test))

370
370


### Convert data from str to list of ints

In [16]:
import string

def Convert_to_list_of_ints(data): 
    t=[]
    c=[]
    for elem in data:
        for i in range(9):
            temp = elem[i].strip(string.punctuation)
            temp = temp.replace(" ", "")
            temp = temp.replace("'","")
            temp = list(map(float, temp.split(",")))
            c.append(temp) #concatenate the sub arrays for each of the 9 lines to make one big array
        t.append(c)
        c=[]
    return t

x_train=Convert_to_list_of_ints(x_train)
x_test=Convert_to_list_of_ints(x_test)

print(len(x_train))
print(len(x_test))


270
370


In [17]:
import numpy as np
import random

# shuffle training data
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

## Turn the lists into np.arrays
x_train, y_train = np.array(x_train), np.array(y_train)

## we also have to convert our data into three-dimensional format
x_train = np.reshape(x_train, (x_train.shape[0], 4,9))

In [18]:
# shuffle testing data
b = list(zip(x_test, y_test))
random.shuffle(b)
x_test, y_test = zip(*b)

## Turn the lists into np.arrays
x_test, y_test = np.array(x_test), np.array(y_test)

## we also have to convert our data into three-dimensional format
x_test = np.reshape(x_test, (x_test.shape[0], 4,9))

### The model

In [35]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, BatchNormalization, Activation
from sklearn.model_selection import KFold
from keras.regularizers import L1L2

def create_model():
    model = Sequential()
##model.add(Bidirectional(LSTM(50, activation='relu',return_sequences=True,kernel_regularizer=L1L2(l1=0.0, l2=0.0)), input_shape=(x_train.shape[1], 9)))
    model.add(LSTM(units=100,activation='tanh', recurrent_activation='sigmoid', return_sequences=False,input_shape=(x_train.shape[1], 9),dropout=0.2,recurrent_dropout=0.2)) 
##model.add(BatchNormalization())
##model.add(LSTM(units=90,activation='relu', recurrent_activation='sigmoid', return_sequences=True,dropout=0.34,recurrent_dropout=0.34))
##model.add(LSTM(units=90, activation='relu', recurrent_activation='sigmoid',return_sequences=False,dropout=0.34,recurrent_dropout=0.34))
    model.add(Dense(units=9))
    model.add(Activation("linear")) 
    opt= keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error',metrics = ['accuracy'])
    return model
#model.fit(x_train, y_train, batch_size=27, epochs=2) #was 150

# saving the model
#model.save('keras_ML1.h5')


#FIRST TEST K-FOLD
def train_and_evaluate_model(model, data, labels, Data, Labels):
    model.fit(data, labels, batch_size=27, epochs=3) #was 150 #training subset
    score = model.evaluate(Data, Labels, verbose=0) #testing subset
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return score
    # fit and evaluate here.

Test_Loss = []
Test_Accuracy = []
    
from sklearn.model_selection import KFold
#if __name__ == "__main__":
data, labels = x_train, y_train
kf = KFold(n_splits=10) 
i=0
for train, test in kf.split(x_train):
    print("%s %s" % (train, test))
    i=i+1
    print("Running Fold", i,"/", 10)
    model = None # Clearing the NN.
    model = create_model()
    score = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
    Test_Loss.append(score[0])
    Test_Accuracy.append(score[1])

[ 27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269] [ 0  1  2  3 

In [36]:
Test_Accuracy

[0.8148148059844971,
 0.7777777910232544,
 0.6296296119689941,
 0.8148148059844971,
 0.8888888955116272,
 0.6666666865348816,
 0.7407407164573669,
 0.7037037014961243,
 0.6666666865348816,
 0.7777777910232544]

In [37]:
Test_Loss

[0.06380745023488998,
 0.06770960241556168,
 0.06912577897310257,
 0.06650032848119736,
 0.0608568973839283,
 0.07028894871473312,
 0.0662907287478447,
 0.06565696001052856,
 0.06806913018226624,
 0.061138901859521866]

In [13]:
#experiment cell, delete after
#Try k-fold 21st January from https://scikit-learn.org/stable/modules/cross_validation.html

def train_and_evaluate__model(model, data[train], labels[train], data[test], labels[test]):
    history = model.fit(x_train, y_train, batch_size=27, epochs=2) #was 150
    # fit and evaluate here.

from sklearn.model_selection import KFold
#if __name__ == "__main__":
data, labels = x_train, y_train
kf = KFold(n_splits=10)   
for train, test in kf.split(x_train):
    print("%s %s" % (train, test))
    print "Running Fold", "/", 10
    model = None # Clearing the NN.
    model = create_model()
    train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])

[ 27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269] [ 0  1  2  3 

In [ ]:
#old
def train_and_evaluate__model(model, data[train], labels[train], data[test], labels[test]):
    history = model.fit(x_train, y_train, batch_size=27, epochs=150)
    # fit and evaluate here.

#Try k-fold 21st January from https://scikit-learn.org/stable/modules/cross_validation.html
x_train, y_train
>>> kf = KFold(n_splits=2)
>>> for train, test in kf.split(x_train):
...     print("%s %s" % (train, test))


#https://github.com/keras-team/keras/issues/1711
if __name__ == "__main__":
    data, labels, header_info = load_data()
    kf = KFold(labels, n_splits=10, shuffle=True)

    for i, (train, test) in enumerate(kf):
            print "Running Fold", i+1, "/", n_folds
            model = None # Clearing the NN.
            model = create_model()
            train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test))

In [7]:
from keras.models import load_model

# testing the model
model = load_model('keras_ML1.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.041191194548800186
Test accuracy: 0.8243243098258972
